●optimizerのまとめ

勾配を計算するoptimizerについて、色々あるけどしょっちゅう忘れるのでメモ
基本的に下記の論文をそのまま日本語にした感じ。

[Incorporating Nesterov Momentum into Adam](http://cs229.stanford.edu/proj2015/054_report.pdf)

なお、optimizerはモデルやアプリケーションによって最適なものが異なるので、一概に「これを使えばOK」と言ったものはまだない。
また、実務で機械学習をするにあたり、**「目的を達成するためにはoptimizerをどうこうするよりもアーキテクチャーを変える方が圧倒的に有効」**である点は忘れてはいけない。

# 1.勾配降下法
大まかに下記の３つがある。


### 1-1.最急降下法（Gradient Descent)   
最も単純な勾配法。  
難しく考えなくていい、数式の通り「勾配方向に進む」形でパラメータを更新するだけ。  
その心は、「勾配ベクトルがコスト関数を小さくする方向を示してくれているから」というもの。    

  
更新する重みパラメータをW、学習率をη、コスト関数をLとすると、重みWを更新する式は下記の通り。  
（←は右の式で左を更新する、という意味）  

$$g_t =  - \eta_{t-1}\frac{\partial L}{\partial W_{t-1}}$$  
$$W_t ← W_{t-1} + g_t$$


上記の数式を日本語にすると下記のイメージ
$$重みパラメータ_t　← 重みパラメータ_{t-1} + (学習率_{t-1}*勾配ベクトル_{t-1})$$

端的に言うと「勾配ベクトルに学習率をかけたものを足し合わせる」ような更新をしている。  
学習率にどのような値を設定するかが肝で、大きくすれば毎回の重み更新で大きく値を更新できるので早く学習が収束するかもしれないが、  
一方、いつまでも収束しない可能性もある。

##### メリット
アルゴリズムが理解しやすく、実装が単純

##### デメリット
全てのデータの誤差を計算した後にパラメータを更新するので、計算量が多い。
また、コスト関数が複雑な形をしていると局所解に陥りやすい。  
（鞍点にハマりやすい）

##### 余談 
勾配ベクトルが「コスト関数を小さくする」方向を示しているものの、負の値となっているので、−1をかけて正しく「コスト関数を小さくする方向」の勾配ベクトルを足し合わせている。  
学習率（η）をハイパーパラメータとして設定する必要があるが、これの調整が難しい。

##### 参照
[wiki](https://ja.wikipedia.org/wiki/%E6%9C%80%E6%80%A5%E9%99%8D%E4%B8%8B%E6%B3%95)  




### 1-2.確率的勾配降下法（Stochastic Gradient Descent - SDG）  
基本的には「最急降下法」と同じ。  
SGDは「訓練データを1つだけ取り出して誤差を計算し、パラメーターを更新する」というものなので「最急降下法」よりも計算量が少ない。  

結構よく使われているみたいです。


### 1-3.ミニバッチ確率的勾配降下法（Minibatch SGD - MSGD）  
「最急降下法」と「確率的勾配降下法」の中間のようなイメージ。  
あらかじめ決めたサイズ（バッチサイズという）で誤差を計算して、パラメータを更新する

ディープラーニングの最適化でもミニバッチはよく使われているみたいですね。

# 2.Momentum  

### 2-1.普通のMomentum
ほとんどが「勾配降下法」と同じで、「慣性」のような概念が追加されたイメージ。  
ここで言う「慣性」とは、「前回大きくパラメータを更新したなら、次も大きく更新する」といったもの。  
（逆に言うと、更新分が小さいようなら次の更新分も小さくなる）  
数式的に言うと下記。  

$$g_t ← \frac{\partial L}{\partial W_{t-1}}$$  
$$m_t ←  µm_{t−1} + g_t $$  
$$W_t ← W_{t-1}- \eta m_t$$

mが「前回の更新量をどれだけ次回の更新に反映させるか」といったニュアンスの「慣性」の位置付け。  
なので、**「今まで更新し続けたパラメータは次回も更新しやすい」**ものになる

### 2-2.（Nesterov’s Accelerated Gradient - NAG）
ちょっと複雑だし、あまり使われていない。  

$$g_t ← \frac{\partial L}{\partial (W_{t-1}-ηµm_{t-1})}$$  
$$m_t ← µm_{t−1} +g_t$$  
$$W_t ← W_{t−1} −ηm_t$$


### 2-3.勾配降下法との違い

・慣性のような概念が追加されたので「SGDのようなジグザグしたパラメータ探索」をしなくなる  
・$m*g_{t-1}$の分だけ重みパラメータの更新量が大きくなる。  
（SGDと比較すると、学習率を少し下げた方がいいかもしれません）  
・

### 2-4.Momentum SGD
MomentumとSGDの概念を組み合わせたoptimizerもよく使われているみたいですね。
基本的には上記の「Momentum」の計算式に基づきますが、「訓練データ１つから、誤差を計算しパラメータを更新する」というフローになる。


### 2-5.メリット
学習の収束化が勾配降下法よりも早い、らしいが実証できていないし数式的な証明も見ていないのでよくわからない。

### 2-6.デメリット


### 余談
µを大きく設定すると「慣性が強くなる」イメージなので「新しいパラメータの移動に対して鈍感」になる。  
基本的にµは１より小さい値を設定する。
調整するパラメータが２つ（η、µ）あるので、パラメータチューニングが難しい。

# 3.AdaGrad  

### 3-1.概要
過去の勾配の値を累積し、それに基づいて全体的に学習を適応させる、といったもの。  
つまり、学習係数を自動で調整してくれる。  
$g_t$は勾配ベクトルで、$g_t^2$はアダマール積。
$r$が勾配を蓄積するベクトル。

$$g_t ← \frac{\partial L}{\partial W_{t-1}}$$  
$$r_t ← r_{t-1} + g_t^2$$  
$$W_i ← W_{i-1} - η\frac{g_t}{δ + r_t^{1/2}}$$


δは分母が０にならないようにとても小さい値を足してあげる。  
($10^{-7}とか$）


### 3-2.パラメータ
η:学習率  
処理中はいい感じに上記の式で調整してくれるが、とはいえ最初はちゃんと設定してあげる必要がありますよ。

### 3-3.余談
$δ + r_t^{1/2}$はブロードキャストされた状態で$g_t$に各要素ごとに適用される。  
（アダマール積）

数式を見ると、勾配が大きいパラメータの学習率を急速に減少させる傾向がある。  
それに付随してか、AdaGradは早まって学習率を減少させてしまう傾向があるそうです。  
この欠点の解消を目指したオプティマイザが「RMSProp」らしい。
（参考文献[実践Deep Learning](https://www.amazon.co.jp/%E5%AE%9F%E8%B7%B5-Deep-Learning-%E2%80%95Python%E3%81%A8TensorFlow%E3%81%A7%E5%AD%A6%E3%81%B6%E6%AC%A1%E4%B8%96%E4%BB%A3%E3%81%AE%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%82%A2%E3%83%AB%E3%82%B4%E3%83%AA%E3%82%BA%E3%83%A0-%E3%82%AA%E3%83%A9%E3%82%A4%E3%83%AA%E3%83%BC%E3%83%BB%E3%82%B8%E3%83%A3%E3%83%91%E3%83%B3/dp/4873118328)

# 4.RMSProp

### 4-1.概要
Adagradでは、勾配が大きいパラメータの学習率を急速に減少させる傾向があるため、極小値に陥りやすい。  
これへの対応として、AdaGradに「古い勾配の影響度を小さくする」概念を追加したアルゴリズムを**RMSProp**という。  
数式的は下記の通りで、**AdaGradとの違いは、二つ目の式の部分だけ**。  


$$g_t ← \frac{\partial L}{\partial W_{t-1}}$$  
$$r_t ← ρr_{t-1} + (1- ρ)g_t^2$$  
$$W_i ← W_{i-1} - η\frac{g_t}{δ + r_t^{1/2}}$$



AdaGradとは異なり、直近の勾配の影響をより重視したoptimizerです。  
ρは「減衰係数」と言われ、大きいほど古い勾配の値を保持する。

# 5.Adam（adaptive moment estimation)

### 5-1.概要
MomentumとRMSPropを組み合わせたようなアルゴリズム。  
2015年に発表された、割と新しいアルゴリズム。


$g_t ← \frac{\partial L}{\partial W_{t-1}}$・・・①いつも通りの勾配計算  
$m_t ← µm_{t−1} + (1− µ)g_t$・・・②勾配に対してのモーメンタム保持  
$\hat{m_t} ← \frac{m_t}{1−µ^t}$・・・③モーメンタムを少し修正  
  
$r_t ← ρr_{t−1} + (1−ρ)g_t^2$・・・④RMSProp的に、勾配を保持  
$\hat{r_t} ← \frac{r_t}{(1−ρ^t)}$・・・⑤RMSPropの値を少し修正  
$W_t ← W_{t−1} −η\frac{\hat{m_t}}{ \hat{r_t}^{1/2}+ε}$・・・⑥重みパラメータ更新(③、⑤を利用)  


### 5-2.指定するパラメータ
下記の3つ。

µ:momentum係数。１以下に設定することで収束させることが基本。（③の式を見ればわかるが、これで収束させる）  
ρ:減衰係数１以下に設定することで収束させることが基本。（⑤の式を見ればわかるが、これで収束させる）  
ε:分母が0にならないように足し合わせてあげる数字。$10^{-7}$とかでいいんじゃないでしょうか？
